## Project – Creating an automated trading system

#### Christian Karvonen IA-15 & Wilhelm Kinos IA-15

#### Imports:

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LinearRegression, ElasticNetCV, Ridge, BayesianRidge, LassoCV, LassoLarsCV
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, svm, model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
init_notebook_mode(connected=True)

### All definitions/functions:

#### Rolling average definition:

In [2]:
def rolling_average(values, window):
    nan = []
    nan = np.append(nan, np.repeat(np.nan, window-1))
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values,weights,'valid')
    sma = np.append(nan, sma)
    return sma

#### MASE

In [3]:
def mean_absolute_scaled_error(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    n = len(y_test)
    d = np.abs(np.diff(y_test, axis=0)).sum()/(n-1) 
    errors = np.abs(y_test.flat - forecast_prediction)
    return errors.mean()/d

#### SMAPE

In [4]:
def SMAPE(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    return 1/len(y_test) * np.sum(np.abs(forecast_prediction - y_test.flat) / (np.abs(y_test) + np.abs(forecast_prediction))/2)

#### Defining timeframe of stocks:

In [5]:
end = dt.date.today()
endday = end
endday = endday.strftime('%Y-%m-%d')
print([endday])
start = end - dt.timedelta(days=5*365)

['2018-10-30']


#### Use of DataReader to get MU stock data from IEX https://iextrading.com/developer/

In [6]:
df = DataReader('MU', 'iex', start, end)
df.tail(10)

,open,high,low,close,volume
date,,,,,
2018-10-16,42.70,43.3500,42.30,43.21,28827416
2018-10-17,43.33,43.3900,42.11,42.35,27758963
2018-10-18,42.04,42.2125,41.14,41.30,33560822
2018-10-19,41.67,41.9850,40.27,40.45,28641375
2018-10-22,40.56,40.7000,38.64,39.76,34441221
2018-10-23,38.15,39.1100,37.41,38.68,46364174
2018-10-24,38.00,38.3000,35.40,35.43,53096450
2018-10-25,35.59,37.1600,35.06,36.78,39625699
2018-10-26,35.28,35.9100,34.30,35.40,41257357


### The 5 features:

#### Momentum: 

In [7]:
df['momentum'] = df.close - df.close.shift(4)
df[['momentum']].head(10)

,momentum
date,
2013-10-31,NaN
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,-0.180
2013-11-07,0.144
2013-11-08,0.215
2013-11-11,0.600
2013-11-12,1.130


#### A/O Oscillator (accumulation/distribution oscillator):

In [8]:
df['A/O'] = (df.high - df.close.shift(1))/(df.high - df.low)
df[['A/O']].head(10)

,A/O
date,
2013-10-31,NaN
2013-11-01,0.479167
2013-11-04,0.882979
2013-11-05,-0.333333
2013-11-06,0.348837
2013-11-07,1.139130
2013-11-08,0.786275
2013-11-11,0.460526
2013-11-12,0.727273


#### MA5:

In [9]:
df['MA5'] = rolling_average(df.close, 5)
df[['MA5']].head(10)

,MA5
date,
2013-10-31,NaN
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,17.6630
2013-11-07,17.6708
2013-11-08,17.7768
2013-11-11,17.8528
2013-11-12,18.0448


#### BIAS6:

In [10]:
df['MA6'] = rolling_average(df.close, 6)
df['BIAS6'] = ((df.close-df.MA6)/(df.MA6))*100
df[['BIAS6']].head(10)

,BIAS6
date,
2013-10-31,NaN
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,NaN
2013-11-07,0.264066
2013-11-08,2.039283
2013-11-11,2.301361
2013-11-12,3.601683


#### ROC (rate of change)

In [11]:
df['ROC'] = (df.close - df.close.shift(3)) / df.close.shift(3) * 100
df[['ROC']].head(10)

,ROC
date,
2013-10-31,NaN
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,-0.056561
2013-11-06,-0.426743
2013-11-07,-0.955841
2013-11-08,2.461800
2013-11-11,4.400000
2013-11-12,5.141374


### Moving window

In [12]:
def get_list_of_moving_window(data, window):
    return list(zip(*(data[i:] for i in range(window))))

In [15]:
df["window_3"] = pd.Series(get_list_of_moving_window(df["close"], 3), index=df.index[3-1:])
df.head()

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,ROC,window_3
date,,,,,,,,,,,,
2013-11-07,17.83,18.155,17.58,17.719,54196790,0.144,1.139130,17.6708,17.672333,0.264066,-0.955841,NaN
2013-11-08,17.70,18.120,17.61,18.105,33201332,0.215,0.786275,17.7768,17.743167,2.039283,2.461800,NaN
2013-11-11,18.04,18.280,17.90,18.270,36990366,0.600,0.460526,17.8528,17.859000,2.301361,4.400000,"(17.719, 18.105, 18.27)"
2013-11-12,18.17,18.670,18.12,18.630,32232323,1.130,0.727273,18.0448,17.982333,3.601683,5.141374,"(18.105, 18.27, 18.63)"
2013-11-13,18.47,18.990,18.42,18.920,38821218,1.201,0.631579,18.3288,18.190667,4.009382,4.501519,"(18.27, 18.63, 18.92)"


Dropna

In [14]:
df.dropna(inplace=True)
df.head()

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,ROC,window_3
date,,,,,,,,,,,,
2013-11-07,17.83,18.155,17.58,17.719,54196790,0.144,1.139130,17.6708,17.672333,0.264066,-0.955841,"(17.67, 17.5, 17.719)"
2013-11-08,17.70,18.120,17.61,18.105,33201332,0.215,0.786275,17.7768,17.743167,2.039283,2.461800,"(17.5, 17.719, 18.105)"
2013-11-11,18.04,18.280,17.90,18.270,36990366,0.600,0.460526,17.8528,17.859000,2.301361,4.400000,"(17.719, 18.105, 18.27)"
2013-11-12,18.17,18.670,18.12,18.630,32232323,1.130,0.727273,18.0448,17.982333,3.601683,5.141374,"(18.105, 18.27, 18.63)"
2013-11-13,18.47,18.990,18.42,18.920,38821218,1.201,0.631579,18.3288,18.190667,4.009382,4.501519,"(18.27, 18.63, 18.92)"
